In [ ]:
# !pip install fosforml

In [1]:
from fosforml.model_manager.snowflakesession import get_session
session = get_session()

In [2]:
from fosforml.model_manager import snowflakesession
session_instance = snowflakesession()
params = session_instance.connection_params
print(params)

{'user': 'SUNNYCHUGH', 'password': 'Password@1209', 'account': 'rwb76633', 'role': 'SUNNYCHUGH', 'warehouse': 'FOSFOR_WAREHOUSE', 'database': 'INSIGHT_DESIGNER_SPCS', 'schema': 'INSIGHT_SPCS_SCHEMA'}


#### Snowflake Modeling

In [3]:
from snowflake.snowpark.session import Session,col
import snowflake.snowpark.functions
from snowflake.ml.modeling.metrics import confusion_matrix, accuracy_score, f1_score, recall_score,precision_score
from snowflake.ml.modeling.preprocessing import LabelEncoder, StandardScaler
from snowflake.ml.modeling.impute import SimpleImputer
from snowflake.ml.modeling.ensemble import GradientBoostingClassifier
from snowflake.ml.modeling.pipeline import Pipeline
import snowflake

In [4]:
def apply_label_encoding(input_data_frame):
    for i in input_data_frame.dtypes:
        if i[1].find('string') >= 0:
            label_encoder = LabelEncoder(input_cols=i[0],output_cols=i[0],drop_input_cols=True)
            input_data_frame = label_encoder.fit(input_data_frame).transform(input_data_frame)
    return input_data_frame

In [5]:
# snow_df = session.table("HR_CHURN")
import pandas as pd
df =pd.read_csv("HR_CHURN.csv")
snow_df = session.create_dataframe(df)
input_data_frame, test_df = snow_df.randomSplit([0.75, 0.25])

ProgrammingError: 090073 (22000): Warehouse 'FOSFOR_WAREHOUSE' cannot be resumed because resource monitor 'FOSFOR_RM' has exceeded its quota.

In [7]:
input_data_frame = input_data_frame.drop("Unnamed: 0")
test_df = test_df.drop("Unnamed: 0")

In [9]:
label_encoder_column = []     
input_data_frame = apply_label_encoding(input_data_frame)
feature_cols = input_data_frame.columns
target_col = "SALARY"
feature_cols.remove(target_col)
OUTPUT_COLS = [target_col + '_PREDICTION']

In [10]:
input_data_frame.show()

----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|"SALARY"  |"DEPARTMENT"  |"SATISFACTION_LEVEL"  |"LAST_EVALUATION"  |"NUMBER_PROJECT"  |"AVERAGE_MONTLY_HOURS"  |"TIME_SPEND_COMPANY"  |"WORK_ACCIDENT"  |"LEFT"  |"PROMOTION_LAST_5YEARS"  |
----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|1.0       |7.0           |0.38                  |0.53               |2                 |157                     |3                     |0                |1       |0                        |
|2.0       |7.0           |0.11                  |0.88               |7                 |272                     |4                     |0                |1       |0                        |
|1.0       |7.0           |0.72              

In [12]:
pipeline = GradientBoostingClassifier(input_cols=feature_cols, label_cols=target_col)
pipeline.fit(input_data_frame)

/opt/conda/lib/python3.9/site-packages/sklearn/base.py:348: InconsistentVersionWarning: Trying to unpickle estimator DummyClassifier from version 1.3.0 when using version 1.3.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/opt/conda/lib/python3.9/site-packages/sklearn/base.py:348: InconsistentVersionWarning: Trying to unpickle estimator DecisionTreeRegressor from version 1.3.0 when using version 1.3.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/opt/conda/lib/python3.9/site-packages/sklearn/base.py:348: InconsistentVersionWarning: Trying to unpickle estimator GradientBoostingClassifier from version 1.3.0 when using version 1.3.2. This might lead to 

In [10]:
pipeline.label_cols

['SALARY']

In [11]:
!pip freeze | grep -i sci

scikit-learn==1.3.2
scipy==1.13.1


In [13]:
test_df = apply_label_encoding(test_df)

In [14]:
pred_df = pipeline.predict(test_df)

The version of package 'scikit-learn' in the local environment is 1.3.2, which does not fit the criteria for the requirement 'scikit-learn==1.3.0'. Your UDF might not work when the package version is different between the server and your local environment.


In [15]:
pred_df.show()

----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|"SALARY"  |"DEPARTMENT"  |"SATISFACTION_LEVEL"  |"LAST_EVALUATION"  |"NUMBER_PROJECT"  |"AVERAGE_MONTLY_HOURS"  |"TIME_SPEND_COMPANY"  |"WORK_ACCIDENT"  |"LEFT"  |"PROMOTION_LAST_5YEARS"  |"OUTPUT_SALARY"  |
----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|2.0       |7.0           |0.8                   |0.86               |5                 |262                     |6                     |0                |1       |0                        |1.0              |
|1.0       |7.0           |0.37                  |0.52               |2                 |159                     |3                     |0                |1       |

#### Model Registration,

In [16]:
from fosforml import register_model

In [17]:
register_model(
  model_obj=pipeline,
  session=session,
  name="SAURABH_HR_CHURN_17July",
  snowflake_df=pred_df,
  dataset_name="HR_CHURN",
  dataset_source="SnowflakeDataset",
  description="This is a test model",
  flavour="snowflake",
  model_type="classification",
  conda_dependencies=["scikit-learn==1.3.2"]
)

Calculating build time metrics

Progress: ███████████████████████                                                33.3%


The version of package 'scikit-learn' in the local environment is 1.3.2, which does not fit the criteria for the requirement 'scikit-learn<1.4'. Your UDF might not work when the package version is different between the server and your local environment.
DataFrame.flatten() is deprecated since 0.7.0. Use `DataFrame.join_table_function()` instead.
The version of package 'scikit-learn' in the local environment is 1.3.2, which does not fit the criteria for the requirement 'scikit-learn<1.4'. Your UDF might not work when the package version is different between the server and your local environment.
The version of package 'scikit-learn' in the local environment is 1.3.2, which does not fit the criteria for the requirement 'scikit-learn<1.4'. Your UDF might not work when the package version is different between the server and your local environment.


Calculating build time metrics

Progress: ███████████████████████████████████████████████                        66.7%
Calculating build time metrics

Progress: ██████████████████████████████████████████████████████████████████████ 100.0%


/opt/conda/lib/python3.9/contextlib.py:119: UserWarning: `relax_version` is not set and therefore defaulted to True. Dependency version constraints relaxed from ==x.y.z to >=x.y, <(x+1). To use specific dependency versions for compatibility, reproducibility, etc., set `options={'relax_version': False}` when logging the model.
  return next(self.gen)
/opt/conda/lib/python3.9/site-packages/snowflake/ml/model/_packager/model_packager.py:92: UserWarning: Inferring model signature from sample input or providing model signature for Snowpark ML Modeling model is not required. Model signature will automatically be inferred during fitting. 
  handler.save_model(


Failed to set tags for model MODEL_216F1A62_8359_453E_AB91_1D36E047B5A9_FDC_SAURABH_HR_CHURN_17JULY.
(1304): 01b5b90e-0002-a410-0033-ac8700a3a026: 000002 (0A000): Unsupported feature 'TAG'.


"Model 'MODEL_216F1A62_8359_453E_AB91_1D36E047B5A9_FDC_SAURABH_HR_CHURN_17JULY' registered successfully."

In [16]:
session.sql("SELECT TO_CHAR(CURRENT_TIMESTAMP,'YYYY-MM-DD HH12:MI:SS PM')").to_pandas().squeeze()

'2024-07-16 07:52:42 AM'

In [17]:
current_data = pipeline.predict_proba(test_df)
predict_col = list(pipeline.predict(test_df).to_pandas()['OUTPUT_SALARY'].to_numpy())
from snowflake.snowpark.functions import lit
current = None
for i in range(len(predict_col)):
    current = current_data.with_column('OUTPUT_SALARY',lit(predict_col[i]))

In [18]:
current_pandas = current.to_pandas()
current_pandas.head()

,SALARY,DEPARTMENT,SATISFACTION_LEVEL,LAST_EVALUATION,NUMBER_PROJECT,AVERAGE_MONTLY_HOURS,TIME_SPEND_COMPANY,WORK_ACCIDENT,LEFT,PROMOTION_LAST_5YEARS,"""PREDICT_PROBA_0.0""","""PREDICT_PROBA_1.0""","""PREDICT_PROBA_2.0""",OUTPUT_SALARY
0,1.0,2.0,0.38,0.50,2,132,3,0,1,0,0.027671,0.535198,0.437131,1.0
1,2.0,6.0,0.90,0.98,4,264,6,0,1,0,0.018951,0.644719,0.336330,1.0
2,1.0,7.0,0.11,0.97,6,284,4,0,1,0,0.018491,0.643398,0.338111,1.0
3,2.0,9.0,0.37,0.48,2,137,3,0,1,0,0.031146,0.551970,0.416884,1.0
4,1.0,5.0,0.40,0.52,2,155,3,0,1,0,0.038289,0.589165,0.372546,1.0


In [19]:
reference_data = pipeline.predict_proba(input_data_frame)
predict_col = list(pipeline.predict(input_data_frame).to_pandas()['OUTPUT_SALARY'].to_numpy())
from snowflake.snowpark.functions import lit
reference = None
for i in range(len(predict_col)):
    reference = reference_data.with_column('OUTPUT_SALARY',lit(predict_col[i]))

In [20]:
reference_pandas = reference.to_pandas()
reference_pandas.head()

,SALARY,DEPARTMENT,SATISFACTION_LEVEL,LAST_EVALUATION,NUMBER_PROJECT,AVERAGE_MONTLY_HOURS,TIME_SPEND_COMPANY,WORK_ACCIDENT,LEFT,PROMOTION_LAST_5YEARS,"""PREDICT_PROBA_0.0""","""PREDICT_PROBA_1.0""","""PREDICT_PROBA_2.0""",OUTPUT_SALARY
0,1.0,7.0,0.37,0.52,2,159,3,0,1,0,0.027898,0.658963,0.313138,1.0
1,1.0,7.0,0.84,0.92,4,234,5,0,1,0,0.020352,0.659914,0.319734,1.0
2,1.0,7.0,0.82,0.87,4,239,5,0,1,0,0.025122,0.662690,0.312188,1.0
3,1.0,9.0,0.41,0.54,2,152,3,0,1,0,0.025384,0.665482,0.309134,1.0
4,1.0,8.0,0.40,0.54,2,148,3,0,1,0,0.024487,0.697544,0.277969,1.0


In [21]:
inference_pandas_data = test_df.to_pandas()

In [22]:
session.write_pandas(reference_pandas,table_name="HR_REFERENCE_DATA",overwrite=True).show()

----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|"SALARY"  |"DEPARTMENT"  |"SATISFACTION_LEVEL"  |"LAST_EVALUATION"  |"NUMBER_PROJECT"  |"AVERAGE_MONTLY_HOURS"  |"TIME_SPEND_COMPANY"  |"WORK_ACCIDENT"  |"LEFT"  |"PROMOTION_LAST_5YEARS"  |"""PREDICT_PROBA_0.0"""  |"""PREDICT_PROBA_1.0"""  |"""PREDICT_PROBA_2.0"""  |"OUTPUT_SALARY"  |
----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|1.0       |7.0           |0.37                  |0.52               |2                 |159                     |3                     |0 

In [23]:
session.write_pandas(current_pandas,table_name="HR_CURRENT_DATA",overwrite=True).show()

----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|"SALARY"  |"DEPARTMENT"  |"SATISFACTION_LEVEL"  |"LAST_EVALUATION"  |"NUMBER_PROJECT"  |"AVERAGE_MONTLY_HOURS"  |"TIME_SPEND_COMPANY"  |"WORK_ACCIDENT"  |"LEFT"  |"PROMOTION_LAST_5YEARS"  |"""PREDICT_PROBA_0.0"""  |"""PREDICT_PROBA_1.0"""  |"""PREDICT_PROBA_2.0"""  |"OUTPUT_SALARY"  |
----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|1.0       |2.0           |0.38                  |0.5                |2                 |132                     |3                     |0 

In [24]:
session.write_pandas(inference_pandas_data,table_name="HR_INFERENCE_DATA",overwrite=True).show()

----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|"SALARY"  |"DEPARTMENT"  |"SATISFACTION_LEVEL"  |"LAST_EVALUATION"  |"NUMBER_PROJECT"  |"AVERAGE_MONTLY_HOURS"  |"TIME_SPEND_COMPANY"  |"WORK_ACCIDENT"  |"LEFT"  |"PROMOTION_LAST_5YEARS"  |
----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|1.0       |7.0           |0.78                  |0.99               |4                 |255                     |6                     |0                |1       |0                        |
|1.0       |7.0           |0.45                  |0.51               |2                 |160                     |3                     |1                |1       |1                        |
|1.0       |7.0           |0.76              